In [ ]:
import json
import uuid
from typing import Dict, Any, List, Tuple, Set

# ===============
# JSInp -> Model
# ===============

def load_jsinp(filename: str) -> Dict[str, Any]:
    with open(filename, 'r', encoding='utf-8') as f:
        return json.load(f)

# ---------- helpers ----------

def _event_is_constant(name: str) -> bool:
    return name in ("<TRUE>", "<FALSE>", "<PASS>")

def _event_is_initiating(evt: Dict[str, Any]) -> bool:
    name = (evt.get('name') or '').strip()
    if name.startswith('IE'):
        return True
    return (evt.get('initf') or '').strip() == 'I'

def _decode_logic_states(logic_val: int, fe_count: int) -> List[str]:
    # LSB-first by FE order: 1=failure, 0=success
    states = []
    for i in range(fe_count):
        bit = (logic_val >> i) & 1
        states.append('failure' if bit == 1 else 'success')
    return states

def _random_model_name() -> str:
    return f"model-{uuid.uuid4().hex[:8]}"


def _decode_fe_states(logic_words: List[Any], fe_count: int, fe_id_to_pos: Dict[int,int]) -> List[str]:
    # Masked-word per-entry decode per dev rule using FE id→position mapping:
    # - Default all FEs to 'bypass'
    # - FE id = (word & 0x3FFFF) (1-based identifier, e.g., 12, 14, 42...)
    # - Position lookup from fe_id_to_pos; ignore ids not present
    # - State: success if word > 2^31, else failure
    states: List[str] = ['bypass'] * fe_count
    THRESH = 1 << 31
    for w in logic_words:
        try:
            val = int(w)
        except Exception:
            continue
        # Determine state first, then derive FE id by stripping highest set bit
        is_success = val > THRESH
        base = (val - THRESH) if is_success else val
        if base <= 0:
            continue
        msb = 1 << (base.bit_length() - 1)
        fe_id = base - msb
        if fe_id <= 0:
            # no second 1 → cannot resolve FE id; leave as bypass
            continue
        pos = fe_id_to_pos.get(fe_id)
        if pos is None or pos < 0 or pos >= fe_count:
            continue
        states[pos] = 'success' if is_success else 'failure'
    return states

# ---------- core builders ----------

def _build_fe_index(sysg: List[Dict[str, Any]]) -> Tuple[List[Tuple[str,int]], Dict[int,int]]:
    fe_order: List[Tuple[str,int]] = []
    fe_id_to_pos: Dict[int,int] = {}
    for fe in sysg:
        fe_name = fe.get('name')
        ftid = fe.get('id')
        try:
            ftid_int = int(ftid)
        except Exception:
            continue
        pos = len(fe_order)
        fe_order.append((fe_name, ftid_int))
        fe_id_to_pos[ftid_int] = pos
    return fe_order, fe_id_to_pos


def build_indices(js: Dict[str, Any]) -> Tuple[Dict[str, Any], Dict[int, Dict[str, Any]], Dict[int, Dict[int, Dict[str, Any]]]]:
    s = js.get('saphiresolveinput', {})
    events = s.get('eventlist', [])
    ev_by_id: Dict[int, Dict[str, Any]] = {}
    for ev in events:
        try:
            ev_by_id[int(ev.get('id'))] = ev
        except Exception:
            continue

    ft_gate_index: Dict[int, Dict[int, Dict[str, Any]]] = {}
    for ft in s.get('faulttreelist', []):
        fth = (ft or {}).get('ftheader', {})
        ftid = fth.get('ftid')
        if ftid is None:
            continue
        try:
            ftid = int(ftid)
        except Exception:
            continue
        gates = {}
        for g in (ft.get('gatelist') or []):
            gid = g.get('gateid')
            if gid is None:
                continue
            try:
                gid = int(gid)
            except Exception:
                continue
            gates[gid] = g
        if gates:
            ft_gate_index[ftid] = gates

    return s, ev_by_id, ft_gate_index


def collect_ft_top_refs(js: Dict[str, Any]) -> Dict[int, Tuple[int, int]]:
    # Returns: ftid -> (gtid, evid)
    res: Dict[int, Tuple[int, int]] = {}
    for ft in js.get('saphiresolveinput', {}).get('faulttreelist', []):
        fth = (ft or {}).get('ftheader', {})
        try:
            ftid = int(fth.get('ftid'))
            gtid = int(fth.get('gtid')) if fth.get('gtid') is not None else None
            evid = int(fth.get('evid')) if fth.get('evid') is not None else None
        except Exception:
            continue
        res[ftid] = (gtid, evid)
    return res


def _list_from_possible_keys(g: Dict[str, Any], suffix: str) -> List[int]:
    # accept keys like 'eventinput', 'eventinputs', 'event_input', etc.
    vals: List[int] = []
    for k, v in g.items():
        if isinstance(k, str) and k.lower().endswith(suffix):
            if isinstance(v, list):
                for x in v:
                    try:
                        vals.append(int(x))
                    except Exception:
                        pass
    return vals


def build_logic_expr_for_gate(ftid: int, gate_id: int, ft_gates: Dict[int, Dict[str, Any]], ev_by_id: Dict[int, Dict[str, Any]]) -> Dict[str, Any]:
    g = ft_gates.get(gate_id, {})
    op = (g.get('gatetype') or 'or').lower()
    if op not in ('and','or','xor','nand','nor','atleast'):
        op = 'or'

    event_ids = _list_from_possible_keys(g, 'eventinput')
    child_gate_ids = _list_from_possible_keys(g, 'gateinput')

    args: List[Dict[str, Any]] = []
    # Recurse gate children first (DFS)
    for cg in child_gate_ids:
        args.append(build_logic_expr_for_gate(ftid, cg, ft_gates, ev_by_id))
    # Then add events
    for eid in event_ids:
        ev = ev_by_id.get(eid)
        if not ev:
            continue
        name = ev.get('id') or f"EV{eid}"
        args.append({ 'event': name })

    # atleast gate may have a k in numinputs or an explicit min
    if op == 'atleast':
        k = g.get('minNumber') or g.get('min') or 0
        try:
            k = int(k)
        except Exception:
            k = 0
        return { 'op': 'atleast', 'k': k, 'args': args }

    # If single arg for and/or, collapse to pass-through
    if op in ('and','or') and len(args) == 1:
        return args[0]
    return { 'op': op, 'args': args }


def build_fault_tree(ftid: int, ft_gates: Dict[int, Dict[str, Any]], top_gate_id: int, ev_by_id: Dict[int, Dict[str, Any]]) -> Tuple[Dict[str, Any], Set[str], Dict[str, bool]]:
    # Returns (FaultTree, used_basic_names, used_house_map)
    name = f"FT{ftid}"
    
    # If top_gate_id exists in gates, use it
    if top_gate_id is not None and top_gate_id in ft_gates:
        top_expr = build_logic_expr_for_gate(ftid, top_gate_id, ft_gates, ev_by_id)
    else:
        # Implement ordered DFS approach: process gates in order, ignoring gtid
        # Find gates that have valid events (filter out gates with non-existent events)
        valid_gates = []
        referenced_gates = set()
        
        for gate_id, gate_data in ft_gates.items():
            # Check if this gate has valid events
            event_ids = _list_from_possible_keys(gate_data, 'eventinput')
            child_gate_ids = _list_from_possible_keys(gate_data, 'gateinput')
            
            # Track which gates are referenced by others
            for child_id in child_gate_ids:
                referenced_gates.add(child_id)
            
            # Check if events exist
            has_valid_events = False
            for eid in event_ids:
                if eid in ev_by_id:
                    has_valid_events = True
                    break
            
            # Gate is valid if it has valid events OR references other gates
            if has_valid_events or child_gate_ids:
                valid_gates.append(gate_id)
        
        # Find root gates (gates not referenced by others)
        root_gates = [gid for gid in valid_gates if gid not in referenced_gates]
        
        if not root_gates:
            # No valid gates found, default to empty OR
            top_expr = { 'op': 'or', 'args': [] }
        elif len(root_gates) == 1:
            # Single root gate, use it directly
            top_expr = build_logic_expr_for_gate(ftid, root_gates[0], ft_gates, ev_by_id)
        else:
            # Multiple root gates, create OR of all roots (ordered DFS)
            args = []
            for gate_id in sorted(root_gates):  # Process in order
                gate_expr = build_logic_expr_for_gate(ftid, gate_id, ft_gates, ev_by_id)
                args.append(gate_expr)
            top_expr = { 'op': 'or', 'args': args }

    # Collect used events for basic/house extraction
    used_basic: Set[str] = set()
    used_house: Dict[str, bool] = {}

    def walk(expr: Dict[str, Any]):
        if not isinstance(expr, dict):
            return
        if 'event' in expr:
            nm = expr['event']
            if nm == '<TRUE>':
                used_house[nm] = True
            elif nm == '<FALSE>':
                used_house[nm] = False
            else:
                used_basic.add(nm)
            return
        if 'op' in expr:
            if expr['op'] == 'not':
                walk(expr.get('arg'))
            else:
                for a in expr.get('args', []):
                    walk(a)
    walk(top_expr)

    # Build basicEvents / houseEvents arrays with available values
    basic_events_arr: List[Dict[str, Any]] = []
    for nm in sorted(used_basic):
        # find value in ev_by_id by name match
        val = None
        for ev in ev_by_id.values():
            if str(ev.get('id')) == str(nm):
                try:
                    val = float(ev.get('value'))
                except Exception:
                    val = None
                break
        if val is None:
            continue
        basic_events_arr.append({ 'name': str(nm), 'p': val })

    house_events_arr: List[Dict[str, Any]] = []
    for nm, st in used_house.items():
        house_events_arr.append({ 'name': nm, 'state': bool(st) })

    ft_obj: Dict[str, Any] = {
        'name': name,
        'top': top_expr
    }
    if basic_events_arr:
        ft_obj['basicEvents'] = basic_events_arr
    if house_events_arr:
        ft_obj['houseEvents'] = house_events_arr
    return ft_obj, used_basic, used_house


def build_event_tree(js: Dict[str, Any], ev_by_id: Dict[int, Dict[str, Any]]) -> Dict[str, Any]:
    hdr = js.get('saphiresolveinput', {}).get('header', {})
    et_meta = hdr.get('eventtree', {})
    et_name = et_meta.get('name') or 'EventTree'

    # Functional events from sysgatelist order
    sysg = js.get('saphiresolveinput', {}).get('sysgatelist', [])
    fe_order, fe_id_to_pos = _build_fe_index(sysg)

    functional_events = []
    for fe_name, ftid in fe_order:
        functional_events.append({ 'name': f"FT{ftid}", 'description': fe_name })

    # Initiating event
    init_id = et_meta.get('initevent')
    initiating = None
    if init_id is not None:
        try:
            init_ev = ev_by_id.get(int(init_id))
            if init_ev:
                initiating = {
                    'name': init_ev.get('name') or f"INIT{init_id}",
                    'frequency': float(init_ev.get('value') or 0.0),
                    'unit': 'yr-1'
                }
        except Exception:
            pass
    if initiating is None:
        # Fallback: detect by name prefix 'IE' or initf == 'I'
        cand = None
        for eid in sorted(ev_by_id.keys()):
            ev = ev_by_id[eid]
            if _event_is_initiating(ev):
                cand = ev
                break
        if cand:
            try:
                initiating = {
                    'name': (cand.get('name') or f"INIT{cand.get('id')}"),
                    'frequency': float(cand.get('value') or 0.0),
                    'unit': 'yr-1'
                }
            except Exception:
                initiating = {
                    'name': (cand.get('name') or f"INIT{cand.get('id')}"),
                    'frequency': 0.0,
                    'unit': 'yr-1'
                }
        else:
            initiating = { 'name': f"INIT{init_id}", 'frequency': 0.0, 'unit': 'yr-1' }

    # Sequences from sequencelist.logiclist
    seqs = []
    sl = js.get('saphiresolveinput', {}).get('sequencelist', [])
    fe_count = len(fe_order)
    for seq in sl:
        seqid = seq.get('seqid')
        logic = seq.get('logiclist') or []
        if not isinstance(logic, list) or len(logic) == 0:
            continue
        states_bits = _decode_fe_states(logic, fe_count, fe_id_to_pos)
        functional_states = []
        for idx, (_fe_name, ftid) in enumerate(fe_order):
            state = states_bits[idx] if idx < len(states_bits) else 'bypass'
            refname = f"FT{ftid}"
            functional_states.append({ 'name': refname, 'state': state })
        seqs.append({
            'name': f"S{seqid}",
            'functionalStates': functional_states,
            'endState': f"S{seqid}"
        })

    et: Dict[str, Any] = {
        'name': et_name,
        'initiatingEvent': initiating,
        'sequences': seqs
    }
    if functional_events:
        et['functionalEvents'] = functional_events
    return et


def convert_jsinp_to_model(js: Dict[str, Any]) -> Dict[str, Any]:
    s, ev_by_id, ft_gate_index = build_indices(js)
    ft_top_refs = collect_ft_top_refs(js)

    # Build FaultTrees in DFS, collect
    fault_trees: List[Dict[str, Any]] = []
    for ftid, (gtid, _evid) in ft_top_refs.items():
        gates = ft_gate_index.get(ftid, {})
        ft_obj, _used_basic, _used_house = build_fault_tree(ftid, gates, gtid, ev_by_id)
        fault_trees.append(ft_obj)

    # Build single EventTree for this JSInp
    event_tree = build_event_tree(js, ev_by_id)

    model: Dict[str, Any] = {
        'faultTrees': fault_trees,
        'eventTrees': [event_tree]
    }

    # Optional name from header.eventtree.name; otherwise random minimal name
    model['name'] = _random_model_name()

    return model


def build_scram_node_options() -> Dict[str, Any]:
    return {
        'mocus': False,
        'bdd': True,
        'zbdd': False,
        'rareEvent': False,
        'mcub': False,
        # number fields intentionally omitted per spec (commented-out in TS only)
        'primeImplicants': False,
        'probability': True,
        'importance': False,
        'uncertainty': False,
        'ccf': False,
        'sil': False,
    }


def convert_jsinp_to_request(js: Dict[str, Any]) -> Dict[str, Any]:
    model = convert_jsinp_to_model(js)
    settings = build_scram_node_options()
    return { 'settings': settings, 'model': model }

# ---------- example usage ----------
# Set filename to your JSInp path then run the cell.
# filename = 'fixtures/models/generic-pwr-model/data/saphsolve/EQK-BIN7_et_Grp-1_24-02-26_15-58-50.JSInp'
# js = load_jsinp(filename)
# req = convert_jsinp_to_request(js)
# print(json.dumps(req, indent=2))

In [ ]:
# Google Colab upload/convert/download
try:
    from google.colab import files  # type: ignore
    _COLAB = True
except Exception:
    _COLAB = False

if _COLAB:
    print("Upload a JSInp file...")
    uploaded = files.upload()
    if uploaded:
        filename = list(uploaded.keys())[0]
        js = json.loads(uploaded[filename].decode('utf-8'))
        req = convert_jsinp_to_request(js)
        output = json.dumps(req, indent=2)
        out_name = filename.rsplit('.', 1)[0] + '_request.json'
        with open(out_name, 'w', encoding='utf-8') as f:
            f.write(output)
        print(f"Converted → {out_name}")
        files.download(out_name)
else:
    print("Not running in Colab. To use file upload, open this notebook in Google Colab.")